In [1]:
##Load in data and do some quick processing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from string import punctuation
import re
import seaborn as sns
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
import time 


df = pd.read_csv('~/ML/NFL/FinalSetTest.csv')
df1 = df.select_dtypes(exclude=['object'])
df2 = df.select_dtypes(['object'])
df1 = df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.replace(np.nan, 10)
yTrain = df1['Yards']
playId = df1['PlayId']
xTrain = df1.drop(['GameId','PlayId', 'Yards'], axis=1)
eps = 10**-10
xTrain = np.log(xTrain - xTrain.min() + eps)
#xTrain = pd.get_dummies(xTrain)



x1 = xTrain.iloc[:22500].to_numpy()
y = yTrain.iloc[:22500].to_numpy()

xTest1 = xTrain.iloc[22500:].to_numpy()
yTest = yTrain.iloc[22500:].to_numpy()

scaler = preprocessing.StandardScaler()
pt = preprocessing.PowerTransformer()

#xNorm = pt.fit_transform(x1)
#xNormTest = pt.transform(xTest1)

xScale = scaler.fit_transform(x1)
xScaleTest = scaler.transform(xTest1)

#poly = PolynomialFeatures(2)
#xPoly = poly.fit_transform(xScale) 

from sklearn.decomposition import PCA

# PCA preprocessing
# pca = PCA(n_components=max(20, xScale.shape[-1]//2))
# pca.fit(xScale)
# xScale = pca.transform(xScale)
# xScaleTest = pca.transform(xScaleTest)

In [2]:
##Define a network 
import torch
import torch.nn as nn 

In [3]:
N, D_in, H, D_out = 64, xScale.shape[1], 100, 1
#N, D_in, H, D_out = 64, 1000, 100, 10

In [4]:
xScale = xScale.astype(float)
y = y.astype(float)

In [5]:
# x = torch.randn(N, D_in)
# y = torch.randn(N, D_out)
x = torch.from_numpy(xScale)
y = torch.from_numpy(y)

In [6]:
print(x.dtype, y.dtype)
x = x.type(torch.float32)
y = y.type(torch.float32)
print(x.dtype, y.dtype)

torch.float64 torch.float64
torch.float32 torch.float32


In [16]:
np.isnan(x).sum()

tensor(0)

In [29]:
model = nn.Sequential(
    nn.Linear(D_in, H), 
    nn.ReLU(), 
    nn.Linear(H, D_out),)

In [30]:
model(x[0])

tensor([1.0225], grad_fn=<AddBackward0>)

In [31]:
loss_fn  = nn.MSELoss(reduction='sum')

In [32]:
learning_rate = 1e-12 

In [37]:
for t in range(2000): 
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    #print(y_pred)
    if t%100==99: 
        print(t, loss.item())
    model.zero_grad()
    
    loss.backward()
    
    with torch.no_grad(): 
        for param in model.parameters(): 
            param -= learning_rate * param.grad

99 21318682624.0
199 21285154816.0
299 21257738240.0
399 21234724864.0
499 21215086592.0
599 21198153728.0
699 21183354880.0
799 21170360320.0
899 21158799360.0
999 21148426240.0
1099 21139105792.0
1199 21130678272.0
1299 21123026944.0
1399 21116049408.0
1499 21109653504.0
1599 21103777792.0
1699 21098362880.0
1799 21093339136.0
1899 21088679936.0
1999 21084340224.0


In [39]:
model(x[1])

tensor([7.3302], grad_fn=<AddBackward0>)